In [0]:
import json

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json, current_timestamp, expr

In [0]:
#connect to newly created Event Hub source1 (movies)
primaryKey_movies = "Endpoint=sb://db-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=86ATi9XElcqYcspgAo8KNdXzMiWsSJbj++AEhA35NY8="
entityPath_movies = "EntityPath=eventhub-movies"

connectionString_movies = primaryKey_movies + ";" + entityPath_movies

ehConf_movies = {}

startingEventPosition_movies = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_movies['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_movies)
ehConf_movies["eventhubs.startingPosition"] = json.dumps(startingEventPosition_movies)

In [0]:
#connect to newly created Event Hub source2 (ratings)
primaryKey_ratings = "Endpoint=sb://db-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=86ATi9XElcqYcspgAo8KNdXzMiWsSJbj++AEhA35NY8="
entityPath_ratings = "EntityPath=eventhub-ratings"

connectionString_ratings = primaryKey_ratings + ";" + entityPath_ratings

ehConf_ratings = {}

startingEventPosition_ratings = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_ratings['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_ratings)
ehConf_ratings["eventhubs.startingPosition"] = json.dumps(startingEventPosition_ratings)

In [0]:
streaming_schema_movies = StructType([StructField("name", StringType(), True),
                                      StructField("year", IntegerType(), True),
                                      StructField("director", StringType(), True),
                                      StructField("writer", StringType(), True),
                                      StructField("star", StringType(), True)
                                     ])

streaming_data_movies = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_movies) \
                             .load()

streaming_data_movies = streaming_data_movies.select(from_json(col("body").cast("string"), streaming_schema_movies)) \
                                             .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                             .select(col('data.*'))   

streaming_data_movies.display()

name,year,director,writer,star
Amar Akbar Anthony,null,null,null,null
The Long Riders,null,null,null,null
The Gods Must Be Crazy,null,null,null,null
Ordinary People,null,null,null,null
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland


In [0]:
streaming_schema_ratings = StructType([StructField("name", StringType(), True),
                                       StructField("rating", StringType(), True),
                                       StructField("score", FloatType(), True)
                                      ])

streaming_data_ratings = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_ratings) \
                             .load()

streaming_data_ratings = streaming_data_ratings.select(from_json(col("body").cast("string"), streaming_schema_ratings)) \
                                               .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                               .select(col('data.*'))   

streaming_data_ratings.display()

name,rating,score
Lagaan,PG,9.2
Andhadhun,PG,8.4
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2
Star Wars: Episode V - The Empire Strikes Back,PG,8.2
Fame,R,6.6
Friday the 13th,R,6.1


Because neither of events has event time column, we will create a processing time column for demo purpose (in real world, most operations are on event time, not the processing time)

In [0]:
streaming_data_movies_timestamp = streaming_data_movies.withColumn("movies_timestamp", current_timestamp())

streaming_data_movies_timestamp.display()

name,year,director,writer,star,movies_timestamp
Amar Akbar Anthony,null,null,null,null,2023-03-08T20:44:49.102+0000
The Long Riders,null,null,null,null,2023-03-08T20:44:49.102+0000
The Gods Must Be Crazy,null,null,null,null,2023-03-08T20:44:49.102+0000
Ordinary People,null,null,null,null,2023-03-08T20:44:49.102+0000
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine,2023-03-08T20:44:49.102+0000
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood,2023-03-08T20:44:49.102+0000
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau,2023-03-08T20:44:49.102+0000
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams,2023-03-08T20:44:49.102+0000
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland,2023-03-08T20:44:49.102+0000


In [0]:
streaming_data_ratings_timestamp = streaming_data_ratings.withColumn("ratings_timestamp", current_timestamp())

streaming_data_ratings_timestamp.display()

name,rating,score,ratings_timestamp
Lagaan,PG,9.2,2023-03-08T20:45:05.559+0000
Andhadhun,PG,8.4,2023-03-08T20:45:05.559+0000
Airplane!,PG,7.7,2023-03-08T20:45:05.559+0000
Caddyshack,R,7.3,2023-03-08T20:45:05.559+0000
Friday the 13th,R,6.4,2023-03-08T20:45:05.559+0000
The Blues Brothers,R,7.9,2023-03-08T20:45:05.559+0000
Raging Bull,R,8.2,2023-03-08T20:45:05.559+0000
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,2023-03-08T20:45:05.559+0000
Fame,R,6.6,2023-03-08T20:45:05.559+0000
Friday the 13th,R,6.1,2023-03-08T20:45:05.559+0000


In [0]:
# Define watermarks
streaming_movies_watermark = streaming_data_movies_timestamp.selectExpr("name as movie_name", "year", "movies_timestamp") \
                                                            .withWatermark("movies_timestamp", "10 seconds") 

streaming_movies_watermark.display()

movie_name,year,movies_timestamp
Amar Akbar Anthony,null,2023-03-08T20:47:56.555+0000
The Long Riders,null,2023-03-08T20:47:56.555+0000
The Gods Must Be Crazy,null,2023-03-08T20:47:56.555+0000
Ordinary People,null,2023-03-08T20:47:56.555+0000
The Long Riders,1980,2023-03-08T20:47:56.555+0000
Any Which Way You Can,1980,2023-03-08T20:47:56.555+0000
The Gods Must Be Crazy,1980,2023-03-08T20:47:56.555+0000
Popeye,1980,2023-03-08T20:47:56.555+0000
Ordinary People,1980,2023-03-08T20:47:56.555+0000


In [0]:
streaming_ratings_watermark = streaming_data_ratings_timestamp.selectExpr("name", "score", "ratings_timestamp") \
                                                              .withWatermark("ratings_timestamp", "20 seconds")

streaming_ratings_watermark.display()

name,score,ratings_timestamp
Lagaan,9.2,2023-03-08T20:48:07.700+0000
Andhadhun,8.4,2023-03-08T20:48:07.700+0000
Airplane!,7.7,2023-03-08T20:48:07.700+0000
Caddyshack,7.3,2023-03-08T20:48:07.700+0000
Friday the 13th,6.4,2023-03-08T20:48:07.700+0000
The Blues Brothers,7.9,2023-03-08T20:48:07.700+0000
Raging Bull,8.2,2023-03-08T20:48:07.700+0000
Star Wars: Episode V - The Empire Strikes Back,8.2,2023-03-08T20:48:07.700+0000
Fame,6.6,2023-03-08T20:48:07.700+0000
Friday the 13th,6.1,2023-03-08T20:48:07.700+0000


In [0]:
watermark_join = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                    expr(""" 
                                                    movie_name = name AND 
                                                    ratings_timestamp >= movies_timestamp AND 
                                                    ratings_timestamp <= movies_timestamp + interval 1 minutes
                                                    """
                                                        )
                                                 )

watermark_join.display()

In [0]:
watermark_join_left_outer = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                                expr(""" 
                                                                movie_name = name AND 
                                                                ratings_timestamp >= movies_timestamp AND 
                                                                ratings_timestamp <= movies_timestamp + interval 2 minutes
                                                                """
                                                                ),
                                                                "leftOuter"
                                                             )

watermark_join_left_outer.display()

In [0]:
watermark_join_left_semi = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                                expr(""" 
                                                                movie_name = name AND 
                                                                ratings_timestamp >= movies_timestamp AND 
                                                                ratings_timestamp <= movies_timestamp + interval 2 minutes
                                                                """
                                                                ),
                                                                "leftSemi"
                                                             )

watermark_join_left_semi.display()